In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [49]:
def get_url(search_term):
    
    template = 'https://www.amazon.in/s?k=phones+under+15000&crid=JAV8K4GU7W6H&sprefix=phones%2Caps%2C287&ref=nb_sb_ss_ts-doa-p_1_6'
    search_term = search_term.replace(' ' , '+')
    
    #     adding term to url
    url =  template.format(search_term)
    #     adding page query holder
    url += '&page{}'
    
    return url


def extract_record(item):
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    try:
        #Price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
        
    except  AttributeError:
            return
            url = ''
        
    try:
         #Rating
        rating = item.i.text
        #Review
        review_count = item.find('span', 'a-size-base').text
        
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count,url)
    
    return result

def main(search_term):
    
    driver = webdriver.Chrome(executable_path='X:\Python\Web_Crawling_Scraping\AmazonProducts\chromedriver.exe')

    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
    
        for item in results:
            record = extract_record(item)
            
            if record: 
                records.append(record)


    driver.close()
    
#     saving to csv

    with open('results.csv','w', newline='',encoding='utf-8') as f:
        writer=csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [50]:
main('phones under 15000')